In [1]:
import pandas as pd
import numpy as np
import sys
import os
from rdkit import Chem
from rdkit.Chem import MACCSkeys
import pubchempy as pcp
import time

In [ ]:
print(os.getcwd())
os.chdir('../../')
print(os.getcwd())

In [2]:
#set file paths
dataset_dir = "datasets/"
processed_drug_feature_path = dataset_dir + 'processed/drug/'


drug_chem_info_smiles_file = processed_drug_feature_path + 'drug_chemical_info.tsv'
drug_chem_smiles_maccskeys_info_file = processed_drug_feature_path + 'drug_chem_smiles_maccskeys_info_file.tsv'

drug_pair_score_file = dataset_dir+ 'drug-comb-db/' + 'drugcombs_scored.csv'
drug_with_pubchemid_pair_score_file = processed_drug_feature_path+ 'drugcombs_scored.tsv'

# Extract pubchem id from pubchem database 

In [4]:
def get_compound_from_drug_name(drug_name):
    #this function retrieves a drug's pubchem compound from its drugname using pubchempy
    global flag
    for i in range(0,5):
        try:
            compounds = pcp.get_compounds(drug_name, 'name')    
            if(len(compounds)>0):
            #return the first compound from the search result
                print(drug_name,compounds[0].cid )
                return compounds[0]
            else:
                return None
        except pcp.PubChemHTTPError:
            print('PubChemHTTPError', drug_name)
            time.sleep(0.3)
            continue
    return None

In [5]:
#extract drug names from synergy score file
drug_synergy_df = pd.read_csv(drug_pair_score_file)
print(drug_synergy_df.shape,drug_synergy_df.columns)

drug_name_list = list(set(list(drug_synergy_df['Drug1'].astype(str).unique()) +\
                     list(drug_synergy_df['Drug2'].astype(str).unique())))



(498865, 8) Index(['ID', 'Drug1', 'Drug2', 'Cell line', 'ZIP', 'Bliss', 'Loewe', 'HSA'], dtype='object')
6245


In [6]:
def extract_pubchemid_smiles_weight_using_pubchempy(drug_name_list, drug_chem_info_smiles_file):
    #for drugs present in drug_name_list find out their pubchem_cid, canonical_smiles and molecular weight
    # and write to drug_chem_info_smiles_file.
    global flag
    drug_name_to_pubchem_cid_smiles_df = pd.DataFrame({'Drug_Name' : drug_name_list})

    drug_name_to_pubchem_cid_smiles_df['pubchem_cid'] = pd.Series(str)       
    drug_name_to_pubchem_cid_smiles_df['canonical_smiles'] = pd.Series(str)   
    drug_name_to_pubchem_cid_smiles_df['molecular_weight'] = pd.Series(str)   
    
    drug_name_to_pubchem_cid_smiles_df.set_index('Drug_Name', inplace= True)

    #get pubchemid, canonical smiles and molecular weight from pubchem for all the drug_names
    print(drug_name_to_pubchem_cid_smiles_df.shape)
    count=0
    for drug_name in drug_name_list:
        print('drug name: ', drug_name)
        pubchem_compound = get_compound_from_drug_name(drug_name)
        count +=1
#         print(count)
        if(pubchem_compound != None):
            drug_name_to_pubchem_cid_smiles_df.at[drug_name,'pubchem_cid']=pubchem_compound.cid
            drug_name_to_pubchem_cid_smiles_df.at[drug_name,'canonical_smiles']=pubchem_compound.canonical_smiles
            drug_name_to_pubchem_cid_smiles_df.at[drug_name,'molecular_weight']=pubchem_compound.molecular_weight
    print(count)
    print(drug_name_to_pubchem_cid_smiles_df.head(), drug_name_to_pubchem_cid_smiles_df.shape)
    
    drug_name_to_pubchem_cid_smiles_df.dropna(inplace=True)
    drug_name_to_pubchem_cid_smiles_df.reset_index(inplace=True)
    
    os.makedirs(os.path.dirname(drug_chem_info_smiles_file), exist_ok=True)
    drug_name_to_pubchem_cid_smiles_df.to_csv(drug_chem_info_smiles_file, sep = '\t', index=False)
    print(drug_name_to_pubchem_cid_smiles_df.shape)

In [7]:
force_run = False
if os.path.exists(drug_chem_info_smiles_file) and force_run==False:
    print('reading from already existing drug_chem_info_smiles_file')
else:
    extract_pubchemid_smiles_weight_using_pubchempy(drug_name_list, drug_chem_info_smiles_file)
    


(6245, 3)
drug name:  NCGC00346475-01
NCGC00346475-01 73265337
drug name:  ANTIBIOTIC AD 32
ANTIBIOTIC AD 32 41744
drug name:  PALMATINE CHLORIDE
PALMATINE CHLORIDE 73442
drug name:  MLN-120B
MLN-120B 9929127
drug name:  otenabant
otenabant 10052040
drug name:  PRASUGREL
PRASUGREL 6918456
drug name:  ZINC117147304
ZINC117147304 73265239
drug name:  Ximelagatran
Ximelagatran 9574101
drug name:  Bupropion hydrochloride
Bupropion hydrochloride 62884
drug name:  LY-294002
LY-294002 3973
drug name:  Nemorubicin
Nemorubicin 65907
drug name:  TELMISARTAN
TELMISARTAN 65999
drug name:  Polydatin
Polydatin 5281718
drug name:  Dorzolamide hydrochloride
Dorzolamide hydrochloride 6918132
drug name:  FRAX1036
FRAX1036 71557891
drug name:  TENIPOSIDE
TENIPOSIDE 452548
drug name:  URACIL
URACIL 1174
drug name:  (13S)-3-hydroxy-13-methyl-7,8,9,11,12,14,15,16-octahydro-6H-cyclopenta[a]phenanthren-17-one
(13S)-3-hydroxy-13-methyl-7,8,9,11,12,14,15,16-octahydro-6H-cyclopenta[a]phenanthren-17-one 6710658
d

Nesbuvir 11561383
drug name:  ZIPRASIDONE HCL
ZIPRASIDONE HCL 219099
drug name:  CYROMAZINE
CYROMAZINE 47866
drug name:  ASPARTAME
ASPARTAME 134601
drug name:  NMS-859
NMS-859 71607189
drug name:  CYCLOSPORIN A
CYCLOSPORIN A 5284373
drug name:  CARGLUMIC ACID
CARGLUMIC ACID 121396
drug name:  XL-147
drug name:  Taladegib
Taladegib 49848070
drug name:  ZOLEDRONIC ACID
ZOLEDRONIC ACID 68740
drug name:  LY-2090314
LY-2090314 10029385
drug name:  Olmutinib
Olmutinib 54758501
drug name:  Telatinib
Telatinib 9808844
drug name:  Cutamesine hydrochloride
Cutamesine hydrochloride 9907323
drug name:  CC-223
CC-223 58298316
drug name:  Vesnarinone
Vesnarinone 5663
drug name:  TRIFLUSAL
TRIFLUSAL 9458
drug name:  c-FMS Inhibitor
c-FMS Inhibitor 11654378
drug name:  CYCLOSPORINE
CYCLOSPORINE 5284373
drug name:  Cromoglycate
Cromoglycate 2882
drug name:  CYPROTERONE ACETATE
CYPROTERONE ACETATE 9880
drug name:  NALOXONE HCL
NALOXONE HCL 5464092
drug name:  TELAPREVIR (VX-950)
TELAPREVIR (VX-950) 3010

Cyproheptadine hydrochloride 13770
drug name:  Disopyramide
Disopyramide 3114
drug name:  80306-38-3
80306-38-3 44250175
drug name:  ABIRATERONE ACETATE
ABIRATERONE ACETATE 9821849
drug name:  Isotretinoin
Isotretinoin 5282379
drug name:  Gentian violet cation
Gentian violet cation 3468
drug name:  Cilliobrevin D
Cilliobrevin D 136257953
drug name:  Deferasirox
Deferasirox 214348
drug name:  GSK837149A
GSK837149A 57519517
drug name:  NALTREXONE HCL
NALTREXONE HCL 5485201
drug name:  GLIPIZIDE
GLIPIZIDE 3478
drug name:  SODIUM DIETHYLDITHIOCARBAMATE TRIHYDRATE
SODIUM DIETHYLDITHIOCARBAMATE TRIHYDRATE 517546
drug name:  Tiagabine hydrochloride
Tiagabine hydrochloride 91274
drug name:  NU7441 (KU-57788)
NU7441 (KU-57788) 11327430
drug name:  Batimastat
Batimastat 5362422
drug name:  YM155 (SEPANTRONIUM BROMIDE)
YM155 (SEPANTRONIUM BROMIDE) 11178236
drug name:  SCOPINE
SCOPINE 1274465
drug name:  NCGC00242478-02
NCGC00242478-02 57519506
drug name:  NCGC00263139-01
NCGC00263139-01 57519536


Azasetron 2264
drug name:  Mitoxantrone
Mitoxantrone 4212
drug name:  METHYCLOTHIAZIDE
METHYCLOTHIAZIDE 4121
drug name:  AS-605240
AS-605240 5289247
drug name:  ETHACRIDINE LACTATE MONOHYDRATE
ETHACRIDINE LACTATE MONOHYDRATE 165457
drug name:  Capmatinib
Capmatinib 25145656
drug name:  NCGC00163424-03
NCGC00163424-03 73265206
drug name:  OLMESARTAN
OLMESARTAN 158781
drug name:  MARIZOMIB
MARIZOMIB 11347535
drug name:  220619-73-8
220619-73-8 9863341
drug name:  RACTOPAMINE HCL
RACTOPAMINE HCL 14010333
drug name:  RWJ-67657
RWJ-67657 3008319
drug name:  Fesoterodine fumarate
Fesoterodine fumarate 9849808
drug name:  Methylrosaniline chloride
Methylrosaniline chloride 11057
drug name:  NITRENDIPINE
NITRENDIPINE 4507
drug name:  Piceatannol
Piceatannol 667639
drug name:  AVL-292
AVL-292 59174488
drug name:  IMIPRAMINE
IMIPRAMINE 3696
drug name:  SHP-099
SHP-099 118238298
drug name:  methylene blue
methylene blue 6099
drug name:  1314241-44-5
1314241-44-5 46931242
drug name:  Conivaptan
Co

Raltegravir 54671008
drug name:  FEXOFENADINE HCL
FEXOFENADINE HCL 63002
drug name:  GLYBURIDE
GLYBURIDE 3488
drug name:  WYE-125132 (WYE-132)
WYE-125132 (WYE-132) 25260757
drug name:  PD318088
PD318088 10231331
drug name:  Nitogenin
Nitogenin 99474
drug name:  SG-00529
SG-00529 11998575
drug name:  CHEMBL256963
CHEMBL256963 25169382
drug name:  PF-04880594
PF-04880594 53356806
drug name:  PFI-4
PFI-4 40642506
drug name:  VU0357121
VU0357121 2296132
drug name:  AZD8055
AZD8055 25262965
drug name:  Apigenin
Apigenin 5280443
drug name:  AZ-191
AZ-191 72716071
drug name:  Nelarabine
Nelarabine 3011155
drug name:  GW-4064X
GW-4064X 9893571
drug name:  A-66
A-66 42636535
drug name:  CCT-251545
CCT-251545 77050682
drug name:  cyclosporin A
cyclosporin A 5284373
drug name:  MELPHALAN
MELPHALAN 460612
drug name:  OXYMATRINE
OXYMATRINE 114850
drug name:  BENZETHONIUM CHLORIDE
BENZETHONIUM CHLORIDE 8478
drug name:  PHA-690509
drug name:  Masoprocol
Masoprocol 71398
drug name:  LY3009120
LY300912

...

TASISULAM 10160238
drug name:  TADALAFIL
TADALAFIL 110635
drug name:  TIRAPAZAMINE
TIRAPAZAMINE 135413511
drug name:  124936-75-0
124936-75-0 16725073
drug name:  177530-93-7
177530-93-7 3203
drug name:  oxybutynin
oxybutynin 4634
drug name:  Tideglusib
Tideglusib 11313622
drug name:  TERAZOSIN HCL
TERAZOSIN HCL 44383
drug name:  Methscopolamine (bromide)
Methscopolamine (bromide) 5459110
drug name:  Ro-51
Ro-51 24883277
drug name:  Akt-I-1,2
Akt-I-1,2 9819282
drug name:  RHEOCHRYSIDIN
RHEOCHRYSIDIN 10639
drug name:  SMI-4a
SMI-4a 1361334
drug name:  Nelfinavir
Nelfinavir 64143
drug name:  CHEMBL3188517
CHEMBL3188517 56645961
drug name:  CHLOROGENIC ACID
CHLOROGENIC ACID 1794427
drug name:  Oxibendazole
Oxibendazole 4622
drug name:  Oridonin
Oridonin 5321010
drug name:  HMSL10076
HMSL10076 73265214
drug name:  BERGAPTEN
BERGAPTEN 2355
drug name:  NVP-AUY922
NVP-AUY922 135539077
drug name:  Org 25935
Org 25935 9862992
drug name:  164178-33-0
164178-33-0 4302963
drug name:  Adapalene
Ada

CHEMBL3348916 73265309
drug name:  LESTAURTINIB
LESTAURTINIB 126565
drug name:  ZINC97975747
ZINC97975747 7059633
drug name:  CALCIFEDIOL
CALCIFEDIOL 5283731
drug name:  Fisetin
Fisetin 5281614
drug name:  NCGC00346481-01
NCGC00346481-01 73265228
drug name:  Novobiocin sodium
Novobiocin sodium 23663939
drug name:  PI-103
PI-103 9884685
drug name:  PRP003-062
PRP003-062 53358901
drug name:  LEVOSULPIRIDE
LEVOSULPIRIDE 688272
drug name:  Withaferin A
Withaferin A 265237
drug name:  CLINOFIBRATE
CLINOFIBRATE 2787
drug name:  MEDROXYPROGESTERONE ACETATE
MEDROXYPROGESTERONE ACETATE 6279
drug name:  FG-4592
FG-4592 11256664
drug name:  Eribulin mesylate
Eribulin mesylate 17755248
drug name:  BEZAFIBRATE
BEZAFIBRATE 39042
drug name:  Brigatinib
Brigatinib 68165256
drug name:  PF-00562271
PF-00562271 16118986
drug name:  PENICILLIN G SODIUM
PENICILLIN G SODIUM 23668834
drug name:  CHEMBL371226
CHEMBL371226 11271909
drug name:  SD-06
SD-06 9865587
drug name:  Clobetasol propionate
Clobetasol pr

SCH-79797 4259181
drug name:  Reboxetine mesylate
Reboxetine mesylate 127150
drug name:  NCGC00263092-01
NCGC00263092-01 11711344
drug name:  BERBERINE HCL
BERBERINE HCL 12456
drug name:  FENRETINIDE
FENRETINIDE 5288209
drug name:  EPZ-015666
EPZ-015666 90241673
drug name:  ESTRAMUSTINE PHOSPHATE SODIUM
ESTRAMUSTINE PHOSPHATE SODIUM 444000
drug name:  URSOLIC ACID
URSOLIC ACID 64945
drug name:  Aminoguanidine hemisulfate
Aminoguanidine hemisulfate 2734952
drug name:  LY2608204
LY2608204 46832368
drug name:  Ezutromid
Ezutromid 25109292
drug name:  CHEMBL3348944
CHEMBL3348944 73265198
drug name:  Antimycin A
Antimycin A 14957
drug name:  PD173955
PD173955 447077
drug name:  9-cis-Retinoic acid
9-cis-Retinoic acid 449171
drug name:  NECROSTATIN-1
NECROSTATIN-1 2828334
drug name:  ANASTROZOLE
ANASTROZOLE 2187
drug name:  GNE-618
GNE-618 66613669
drug name:  TAZAROTENE
TAZAROTENE 5381
drug name:  RG-108
RG-108 702558
drug name:  ZINC208949883
ZINC208949883 73265236
drug name:  Savolitinib


GW788388 10202642
drug name:  chelerythrine
chelerythrine 2703
drug name:  Zebularine
Zebularine 100016
drug name:  Dorsomorphin
Dorsomorphin 11524144
drug name:  SUCCINYLCHOLINE CHLORIDE
SUCCINYLCHOLINE CHLORIDE 22475
drug name:  Oligomycin A
Oligomycin A 5281899
drug name:  EZATIOSTAT
EZATIOSTAT 5310939
drug name:  Phenylephrine hydrochloride
Phenylephrine hydrochloride 5284443
drug name:  RILUZOLE
RILUZOLE 5070
drug name:  Raloxifene hydrochloride
Raloxifene hydrochloride 54900
drug name:  MENADIONE
MENADIONE 4055
drug name:  Bortezomib
Bortezomib 387447
drug name:  GNE-490
GNE-490 46180666
drug name:  METRONIDAZOLE
METRONIDAZOLE 4173
drug name:  PI-3065
PI-3065 24937012
drug name:  Flunarizine
Flunarizine 941361
drug name:  Pantothenic acid
Pantothenic acid 6613
drug name:  ELOXATIN (TN) (SANOFI SYNTHELAB)
ELOXATIN (TN) (SANOFI SYNTHELAB) 24197464
drug name:  SULFATHIAZOLE
SULFATHIAZOLE 5340
drug name:  SODIUM SALICYLATE
SODIUM SALICYLATE 16760658
drug name:  KUC107871N
KUC107871N 

VARESPLADIB (LY315920) 155815
drug name:  GW-441756
GW-441756 9943465
6245
                   pubchem_cid  \
Drug_Name                        
NCGC00346475-01       73265337   
ANTIBIOTIC AD 32         41744   
PALMATINE CHLORIDE       73442   
MLN-120B               9929127   
otenabant             10052040   

                                                     canonical_smiles  \
Drug_Name                                                               
NCGC00346475-01     CCC(C(=O)N1CC(CC1CC2=C(NC3=C2C=CC(=C3)F)C4=C(C...   
ANTIBIOTIC AD 32    CCCCC(=O)OCC(=O)C1(CC(C2=C(C1)C(=C3C(=C2O)C(=O...   
PALMATINE CHLORIDE  COC1=C(C2=C[N+]3=C(C=C2C=C1)C4=CC(=C(C=C4CC3)O...   
MLN-120B            CC1=C(C=CC=N1)C(=O)NC2=C3C(=CC(=C2OC)Cl)C4=C(N...   
otenabant           CCNC1(CCN(CC1)C2=NC=NC3=C2N=C(N3C4=CC=C(C=C4)C...   

                   molecular_weight  
Drug_Name                            
NCGC00346475-01               806.9  
ANTIBIOTIC AD 32              723.6  
PALMATINE CHLORIDE    

In [11]:
drug_name_to_pubchem_cid_smiles_weight_df_using_pubchempy =  pd.read_csv(drug_chem_info_smiles_file,sep='\t')

print(drug_name_to_pubchem_cid_smiles_weight_df_using_pubchempy.shape)

drug_names_without_cids = set(drug_name_list) - \
                            set(list(drug_name_to_pubchem_cid_smiles_weight_df_using_pubchempy['Drug_Name']))
print('drugs without cids: ', len(drug_names_without_cids))

(6146, 4)
drugs without cids:  100


In [12]:
print(drug_name_to_pubchem_cid_smiles_weight_df_using_pubchempy.columns)

Index(['Drug_Name', 'pubchem_cid', 'canonical_smiles', 'molecular_weight'], dtype='object')


# Compute Maccskeys

In [17]:
def computeMACCSKeys(df_drug_smile):
    #compute MACCSKeys for drugs present df_drug_smile  from their canonical smiles.
    df_drug_smile['smilesChemObj'] = df_drug_smile['canonical_smiles'].apply\
                                    (lambda x: Chem.MolFromSmiles(x))
    df_drug_smile['MACCSKeys'] = df_drug_smile['smilesChemObj'].apply(lambda x: MACCSkeys.GenMACCSKeys(x))
    
    #convert MACCSKeys object into 0-1 bitstring
    df_drug_smile['MACCSKeys_bitstring'] = df_drug_smile['MACCSKeys'].\
                                            apply(lambda x: x.ToBitString())
    
    df_drug_smile.drop(['smilesChemObj'], axis=1, inplace = True)
    df_drug_smile['MACCSKeys_bitstring'] = df_drug_smile['MACCSKeys_bitstring'].astype(str)
    print( df_drug_smile['MACCSKeys_bitstring'])
    return df_drug_smile


In [18]:
drug_name_to_pubchem_cid_smiles_weight_df_using_pubchempy = \
                computeMACCSKeys(drug_name_to_pubchem_cid_smiles_weight_df_using_pubchempy)
print(drug_name_to_pubchem_cid_smiles_weight_df_using_pubchempy.columns)

os.makedirs(os.path.dirname(drug_chem_smiles_maccskeys_info_file), exist_ok=True)
drug_name_to_pubchem_cid_smiles_weight_df_using_pubchempy.to_csv(drug_chem_smiles_maccskeys_info_file, sep='\t')

0       0000000000000000000000000000000000000000001000...
1       0000000000000000000000000000000000000000001000...
2       0000000000000000000000000000000000000000000000...
3       0000000000000000000000000000000000000000000000...
4       0000000000000000000000000000000000000010000000...
                              ...                        
6141    0000000000000000000000000100000011000010001000...
6142    0000000000000000000000000000000000000110000000...
6143    0000000000000000000000000010000000000000000000...
6144    0000000000000000000000000000000000000000000000...
6145    0000000000000000000000000000000000000000000000...
Name: MACCSKeys_bitstring, Length: 6146, dtype: object
Index(['Drug_Name', 'pubchem_cid', 'canonical_smiles', 'molecular_weight',
       'MACCSKeys', 'MACCSKeys_bitstring'],
      dtype='object')


In [23]:
drug_name_to_pubchem_cid_smiles_weight_df_using_pubchempy = pd.read_csv(drug_chem_smiles_maccskeys_info_file, sep='\t')
drug_name_to_pubchem_cid_smiles_weight_df_using_pubchempy.at[0, 'MACCSKeys_bitstring']                                                                       

'00000000000000000000000000000000000000000010000000000010000000100100000000010001000101010011110111001100110100110010010011100101110110101111111111101111111111111111110'